# Exploring Trove using the Trove API

### Setting Up
####  Importing the required packages to run the code

In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import json
import ast
from tqdm.auto import tqdm
import plotly.express as px
from typing import List, Tuple, Union, Any
from pandas import DataFrame 
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
API_KEY = ""

#### Functions written to access and visualise the data in the Trove API

In [ ]:
class TroveAPIError(Exception):
    """Custom exception for errors when querying the Trove API."""
    pass


def get_records(inc_ratelimit: bool = False, **kwargs: Any) -> Union[List[dict], Tuple[List[dict], List[int]]]:
    """Perform a single Trove API request search;
        return a JSON result and optional rate limit information"""
    
    response = requests.get(**kwargs)

    if response.status_code != 200:
        raise TroveAPIError(f"Request failed with status {response.status_code}: {response.text}")
    
    try:
        json_response = response.json()
        result = json_response['category'][0]['records']
    except (KeyError, IndexError, TypeError, ValueError) as e:
        raise TroveAPIError(f"Unexpected JSON response: {response.text}") from e
        
    if inc_ratelimit:
            return (
                result, 
                [
                    int(response.headers['X-RateLimit-Remaining-Minute']), 
                    int(response.headers['RateLimit-Reset'])
                ]
            )
    else:
        return result


def get_records_data(url:str, params:dict, api_key:str, all_pages:bool=True) -> List[dict]:
    """Search Trove using the API, return search results"""
    data = []
    seen = set()

    try:
        result = get_records(url=url, headers={"X-API-KEY": API_KEY}, params=params)
        for record in result['article']:
            record_key = record['id']
            if record_key not in seen:
                seen.add(record_key)
                data.append(record)
            else:
                print(f"duplicate skipped: {record_key}")
                
    except KeyError as e:
        if result['n'] == int(0):
            print('No records found with search criteria:')
            for key, val in params.items():
                if key in ['category', 'q'] or key.startswith('l-'):
                    print(f"- {key}: {val}")
            return
        else:
            raise

    if all_pages:
        pbar = tqdm(total = result['total'], initial=len(data))
        while 'next' in result.keys():
            url = result['next']
            result, rate_limit_remaining = get_records(url=url, headers={"X-API-KEY": API_KEY}, inc_ratelimit=True)
            block = []
            if 'article' in result.keys():
                for record in result['article']:
                    record_key = record['id']
                    if record_key not in seen:
                        seen.add(record_key)
                        block.append(record)
                    else:
                        print(f"duplicate skipped: {record_key}")
            data.extend(block)
            pbar.update(len(block))
    
            if rate_limit_remaining[0] == 0:
                print(f"Rate limit reached. Pausing for {rate_limit_remaining[1] + 1} seconds...")
                time.sleep(rate_limit_remaining[1] + 1)
    
        pbar.close()

    print(f"Total results fetched: {len(data)}")
    
    return data


def get_record_year_summary(search_term:str,  start_year:Union[str, int]=0, end_year:Union[str, int]=0, url:str='https://api.trove.nla.gov.au/v3/result', api_key:str=API_KEY) -> dict:
    """Calculate how many articles per year are found with the search term,
    and visualise as a bar chart"""
    params = {
        'category': 'newspaper',
        'q': search_term, 
        'encoding': 'json',
        'n': 1
    }
    if start_year == 0: 
        params['sortby'] = 'dateasc'
        result = get_records(url=url, headers={"X-API-KEY": API_KEY}, params=params)
        start_year = int(result['article'][0]['date'][:4])
    else:
        start_year = int(start_year)

    if end_year == 0: 
        params['sortby'] = 'datedesc'
        result = get_records(url=url, headers={"X-API-KEY": API_KEY}, params=params)
        end_year = int(result['article'][0]['date'][:4])
    else:
        end_year = int(end_year)
    
    print(f"Getting records from: {start_year} to {end_year}")

    counts = {}

    for year in tqdm(range(start_year, end_year)):
        str_year = str(year)
        params['l-decade'] = str_year[:3]
        params['l-year'] = str_year

        rate_limit_remaining = 10
        result, rate_limit_remaining = get_records(url=url, headers={"X-API-KEY": API_KEY}, params=params, inc_ratelimit=True)
        counts[str_year] = result['total']
        if rate_limit_remaining[0] == 0:
            time.sleep(rate_limit_remaining[1] + 1)


    fig = px.bar(x=counts.keys(), y=counts.values(), title=f"Count of Articles Found on Trove Newspapers with search term '{search_term}', {start_year}-{end_year}", labels={'x': 'Year', 'y': 'articles found'})
    fig.show("notebook")
    
    return counts


def get_records_count(url:str, params:dict, api_key:str):
    """Search Trove using the API, return number of search results"""
    data = []
    seen = set()

    try:
        result = get_records(url=url, params=params, headers={"X-API-KEY": API_KEY})

        if result['n'] == int(0):
            print('No records found with search criteria:')
        else:
            print(f"{result['total']:,} records found with search criteria:")
        for key, val in params.items():
                if key in ['category', 'q'] or key.startswith('l-'):
                    print(f"- {key}: {val}")
                
    except TroveAPIError as e:
        print(f"API Error: {e}")
    
    return 


def explode_dictonaries(df:DataFrame, col:str)->DataFrame:
    """Where a dataframe column contains dictionaries, splitting these into seperate columns"""
    
    print(f"Exploding column {col}")
    location = df.columns.get_loc(col)
    new_df = pd.json_normalize(df[col])
    
    for column in new_df:
        new_df.rename(columns={column: col + '_' + column}, inplace=True)

    updated_df = pd.concat(
        [
        df.iloc[:, :location-1], 
        new_df,
        df.iloc[:, location+1:]
        ], axis=1
    )
    return updated_df


def split_trove_title(df:DataFrame, title_col:str)->DataFrame:
    """Split the newspaper title column into name, location, and run"""

    new_df = df
    title_location = new_df.columns.get_loc(title_col)
    new_df.insert(title_location, 'newspaper_run', '')
    new_df.insert(title_location, 'newspaper_state', '')
    new_df.insert(title_location, 'newspaper_city', '')
    new_df.insert(title_location, 'newspaper_title', '')
    
    try:
        if new_df.loc[new_df[title_col].str.count('\(') != 1].shape[0] == 0:
            new_df['newspaper_title'] = new_df[title_col].str.split('(').str[0].str.strip()
            new_df[title_col] = new_df[title_col].str.split('(').str[1].str.strip()
        elif new_df.loc[new_df[title_col].str.count('\(') > 1].shape[0] > 0:
            new_df['temp'] = new_df[title_col].str.split('(')
            new_df['newspaper_title'] = new_df['temp'].apply(lambda x: '('.join(x[:-1]))
            new_df[title_col] = new_df['temp'].apply(lambda x: x[-1])
            new_df.drop(columns=['temp'], inplace=True)
        else:
            print('could not split on bracket')
        new_df['newspaper_run'] = new_df[title_col].str.split(':').str[-1].str.strip(')').str.strip()
        new_df[title_col] = new_df.apply(lambda row: row[title_col].replace(row['newspaper_run'], '').strip(' : )'), axis=1)
        new_df['newspaper_state'] = new_df[title_col].apply(lambda x: x.split(',')[-1].strip().strip('.') if ',' in x else x.strip('.'))
        new_df['newspaper_city'] = new_df.apply(lambda row: row[title_col].replace(row['newspaper_state'], '').strip().strip(','), axis=1)
                
        new_df = new_df.drop(columns=[title_col])

        if 'title_id' in new_df.columns:
            new_df.rename(columns={'title_id': 'newspaper_id'}, inplace=True)
        
    except:
        print('could not divide title column')

    return new_df

### Search Terminology
You can make your search query as simple or as complex as you need. It can include:
- *single word* e.g. `'nationalism'` - will search for articles containing that word
- *multiple words* e.g. `'nationalism federalism'` - will search for articles containing either of those words
- *phrase* e.g. `'"United States"'` - will search for articles containing those words in that order - must be in double quotation marks
- *BOOLEAN* terms (AND, OR, NOT) e.g. `'nationalism AND federalism'`

Trove also has Indicies available that you can include in your search. Below are the ones available for newspapers:
- *publictag*: public tags added by users. Exact match only.
    - formatted as 'publictag:(sydney)' e.g. `'highway OR freeway publictag:(sydney)'`
- *lastupdated*: Find everything that has been added or updated in Trove in a particular time frame. (Updates include text correction but not the addition of tags). Only dates in or after April 2012 are supported.
    - formatted as 'lastupdated:[yyyy-mm-ddT00:00:00Z TO *]' or 'lastupdated:[yyyy-mm-ddT00:00:00Z TO yyyy-mm-ddT00:00:00Z]' e.g. `'nationalism lastupdated:[2012-01-01T00:00:00Z TO *]'`
- *taglastupdated*: Find everything that has been tagged in Trove in a particular time frame. Only includes tags added, not tags deleted. Only dates in or after March 2012 are supported.
    - formatted as 'taglastupdated:[yyyy-mm-ddT00:00:00Z TO *]' or 'taglastupdated:[yyyy-mm-ddT00:00:00Z TO yyyy-mm-ddT00:00:00Z]' e.g. `'nationalism taglastupdated:[2012-01-01T00:00:00Z TO *]'`
- *commentlastupdated*: Find everything that has been commented on in Trove in a particular time frame. Only includes comments added, not comments deleted. Only dates in or after March 2012 are supported.
    - formatted as 'commentlastupdated:[yyyy-mm-ddT00:00:00Z TO *]' or 'commentlastupdated:[yyyy-mm-ddT00:00:00Z TO yyyy-mm-ddT00:00:00Z]' e.g. `'nationalism commentlastupdated:[2017-01-01T00:00:00Z TO *]'`
- *date*: date of publication
    - formatted as 'date:[yyyy-mm-dd TO yyyy-mm-dd]' openended possible eg  'date:[yyyy-mm-dd TO *]', can just use year eg 'date:[* TO YYYY]' e.g. `'nationalism date:[1854 TO 1900]'`
- *has*: Does this item have any tags or comments?
    - formatted as 'has:tags' or 'has:comments' e.g. `'nationalism has:comments'`
- *fulltext*: find items containing an exact word, not the stem
    - eg 'fulltext:Jackes' returns items containing the exact word “Jackes” (not “Jack” or “Jacke”) e.g. `'fulltext:nationalism'`
- *headingsAuthorAbstract*: find newspaper articles matching your search query to within the Title and first four lines
    - formatted as 'headingsAuthorAbstract:y' e.g. `'fulltext:nationalism' headingsAuthorAbstract:y`
 
*Can combine indicies, seperated by a space, e.g.*`"q": "Art has:comments commentlastupdated:[2012-03-07T00:00:00Z TO *]"` *searches for articles related to search term Art that have comments, and those comments were updated from the 7th March 2012 or later*

### Data Overviews
using the `get_record_year_summary()` function created above, we can create a bar chart to see how many articles per year contain our search query. 

We need to include our search query, in single quotation marks. (if you have a phrase, the double quotation marks go inside the single quotation marks). 

There are some optional arguments that we can also include:
- *start_year*: when we want the search to start from (default is to begin with the earliest record found
- *end_year*: when we want the search to go to (default is to end with the latest record found
- *api_key*: a Trove API key. (default is to use the variable `API_KEY` created above. You can run without an API Key, but it is much slower

In [ ]:
white_aus_overview = get_record_year_summary('"white australia"')

In [ ]:
# white_aus_overview

In [ ]:
white_aus_overview = get_record_year_summary('"white australia"')

In [ ]:
nationalism = get_record_year_summary('"nationalism"')

### Collecting Metadata
While we are not allowed to access the full text of newspaper articles in Trove, there is still some interesting things we can do with the metadata the API can provide. We can either run an API search using the Python code, or load a CSV file (i.e. one we have downloaded from the Trove interface, or have been provided from someone else. 

#### Searching with the API
When we use the 'requests' library, we can construct the url to have all the parameters we need, and then get the data back in either XML or JSON format (for this notebook, we will be using JSON). 

We need to start with the beginning of the URL, which we are assigning to the variable `url`. 

In [ ]:
url = 'https://api.trove.nla.gov.au/v3/result'

Then we need to specify our search parameters, which we are assigning to the variable `params`. 

Our search query goes with the key `q`. We can also limit the search with other facets, described below, and there are options for how we want the search to be conducted and the results formatted. 

In [ ]:
params = {
    "category": "newspaper",    # The area of Trove we are searching. This parameter must be included, you can only search one area at a time. 
    "q": '"empire loyalty"',                    # Search query. See options described above. 
    # "l-artType": "newspaper", # filter by record type. newspaper options: 'newspaper', 'gazette'
    # "l-category":             # filter by category. newspaper options: 'Article', 'Advertising', 'Details lists, results, guides', 'Family notices', 'Literature'
    # "l-title": "",            # filter by the newspaper or gazette title id.
    # "l-decade": "",           # filter by publication decade, formatted as YYY (e.g. 199 represents 1990 – 1999).
    # "l-year": "",             # filter by publication year. For newspapers, only available if the decade facet is also applied. formatted as YYYY
    # "l-month": "",            # filter by publication month. Only available if the decade and year facets are also applied. formatted as 1, 2, 3 etc
    # "l-illustrated": "",      # filter by if the newspaper article is illustrated? options: true, false
    # "l-illustrationType": "", # filter by type of illustration for newspaper article
    # "l-wordCount": "",        # filter by newspaper article word count. options: "<100 Words", "100 - 1000 Words", "1000+ Words"
    # "l-state": "",            # filter by state of publication for newspaper article
    # "facet": "all",           # all the parameters beginning with "l-" are limiting by facet. The same parameters can be included as values in the results
    # "include": "",            # newspaper options: tags, comments, lists, years (other options to get article text require NLA permission)
    "sortby": "dateasc",        # options: datedesc, dateasc, relevance
    "bulkHarvest": "true",      # Include this parameter if you intend to harvest a set of records for further processing in your own system. 
    "encoding": "json",         # options: xml, json
    "n": 50,                    # The number of results to return per category. Maximum is 100. Default is 20.
    "reclevel": "full",         # Indicates whether to return a full or brief metadata record. options: full, brief. 
    }                               # 'Brief' provides columns: 'id', 'url', 'heading', 'category', 'title_title', 'title_id', 'date', 'page', 'pageSequence', 'relevance_value', 'relevance_score', 'snippet', 'troveUrl'                               
                                    # 'Full' adds the columns: 'identifier', 'trovePageUrl', 'illustrated', 'wordCount', 'correctionCount', 'tagCount', 'commentCount', 'listCount', 'lastCorrection_by', 'lastCorrection_lastupdated', 'pdf'

We can then take these variables, as well as our API_KEY, and use the function

We have created the functions `get_records_count()` and `get_records_data()`, which use the requests library to conduct a search with the Trove API. Both take the arguments:
- *url*: The base url.
- *params*: The search parameters
- *api_key*: The Trove API key

`get_records_count()` prints out the number of records, and the search terms. 
`get_records_data()`returns the metadata results. It also has the optional argument:
- *all_pages*: If you want to return all pages of the result. default is True.

It's important to first see how large your dataset will be. So first, lets start with only a single page, so we can see how many results there are:

In [ ]:
get_records_count(url=url, params=params, api_key=API_KEY)

Then we can actually run the search: 

In [ ]:
data = get_records_data(url=url, params=params, api_key=API_KEY) 

In [ ]:
# Note: if you have a very large number of records to retrieve, I recommend breaking the search up by year like this: 

# data = []
# for year in range(1837, 2022):
#     print(year)
#     str_year = str(year)
#     params["l-decade"] = str_year[:3]
#     params["l-year"] = str_year
#     try:
#         records = get_records_data(url=url, params=params, api_key=API_KEY)
#         data.extend(records)
#     except TypeError:
#         pass

In [ ]:
data[:3]

We can then turn this result into a 'dataframe', with each record a new row:

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

which we can also save as a csv file for future reference:

In [ ]:
search_name = ""
for key, val in params.items():
    if key in ['category', 'q'] or key.startswith('l-'):
        search_name += '_' + val.replace(' ', '_').replace('"', '').rstrip('_')
search_name

In [ ]:
df.to_csv(f'../data/trove_search{search_name}.csv', index=False)

#### Importing a csv file
The Pandas package can read in a CSV file and turn it into a dataframe. If the file is in a different folder than your notebook, you will need to include the filepath (eg folder names)

In [ ]:
df = pd.read_csv('data/trove_search_white_australia.csv')

In [ ]:
df.head()

#### Cleaning the data
Some columns contain dictionaries or lists, which we might want to split up. But when we read them in from a csv, they can be there as strings or text value. 

Here we go through each column, and check that it is an 'object' (as opposed to integer) and that it's not a dictionary already. if it meets those conditions, we search for if all the values that aren't N/A start with a { (dictionary) or [ (list), and if so, convert them. 

We then check if there are columns where all the strings are numbers, and convert them to integers. 

In [ ]:
for col in df.columns:
    if df[col].dtype == object and not df[col].apply(lambda x: isinstance(x, dict)).sum():
        if df[col].dropna().str[0].isin(['{', '[']).sum() + df[col].isna().sum() == df.shape[0]:
            df[col] = df[col].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
        elif (df[col].dropna().str.isdigit().sum() + df[col].isna().sum()) == len(df):
            print(f"{col} should be an int")

Now that we have done that, we can explode out the dictonaries, making new columns with the key value pairs

In [ ]:
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, dict)).sum():
        df = explode_dictonaries(df, col)


Trove newspaper title columns have the name, location, and run of the title all in one column - we can split those with the function `split_trove_title()`

In [ ]:
df = split_trove_title(df, title_col='title_title').copy()

Sometimes if a record is no longer available, we will still get a record id and links, but the rest of the columns are blanks. Below will check what percentage of a row is N/A, and if its above 80%, we drop the row. 

In [ ]:
df['na_percentage'] = df.isna().sum(axis=1) / df.shape[1] * 100
df = df.loc[df['na_percentage']<=80].copy()

Now we want to add a year column, for our visualisations

In [ ]:
df.insert(df.columns.get_loc('date')+1, 
          'year', 
          df['date'].str[:4].astype(int))

In [ ]:
df.head()

### Visualising the data!
There are many different visualisation libraries available, here we are using [Plotly](https://plotly.com/python/) Express to show some of the kinds of visualisations you can make. 


#### Create a bar chart of a column
To create a stacked bar chart, first we need to accumulate the counts. Fortunately, that's easy to do using the `groupby()` function, and applying the size method

In [ ]:
title_name = 'White Australia'

But if we want accumulated counts, that needs to be done first. Fortunately, that's easy to do using the `groupby()` function, and applying the size method

In [ ]:
counts = df.groupby('newspaper_state').size().reset_index(name='count')

fig = px.bar(counts, x="newspaper_state", y="count", color='newspaper_state', title=f'{title_name} Results by State')
fig.update_traces(hovertemplate='<b>%{x}</b><br>Total: %{y}<extra></extra>')

fig.update_traces(
    hovertemplate='<b>%{x}</b><br>Total: %{y}<extra></extra>'
)

fig.show('notebook')

Another possible bar chart would be looking at the results per newspaper. For this, it is clearer to read when it's a horizontal bar chart, and we may need to adjust the height

In [ ]:
counts = df.groupby(['newspaper_title', 'newspaper_state']).size().reset_index(name='count')

fig = px.bar(counts, x='count', y="newspaper_title", color='newspaper_state', orientation='h', height=500, title=f'{title_name} coverage in newspapers')

fig.show('notebook')


Earlier we looked at how many articles per year there were, but what about the word count? Does this change things?

Names of different colour scales are listed at [https://plotly.com/python/builtin-colorscales/](https://plotly.com/python/builtin-colorscales/)

In [ ]:
counts = df.groupby('year')['wordCount'].sum().reset_index()
fig = px.bar(counts, x='year', y="wordCount", color="wordCount", color_continuous_scale=px.colors.sequential.Agsunset_r, title=f'{title_name} wordcount by year')
fig.show('notebook')

What about a scatterplot? This time we want individual dots for each, so we don't need to aggregate. 

In [ ]:
fig = px.scatter(df, x="year", y="newspaper_state", color="relevance_value", size='wordCount', title=f'{title_name} coverage per state per year, noting relevance and word count')
fig.show('notebook')

We can also create word clouds with the Python package WordCloud and the headline column

In [ ]:
text = " ".join(str(i) for i in df['heading'])
title=f'Word Cloud of Headlines Relating to {title_name}'
stopwords = set(STOPWORDS)

wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

fig = px.imshow(wordcloud)
fig.update_layout(
    title=title,
    xaxis_visible=False,
    yaxis_visible=False, 
)
fig.show('notebook')


In [ ]:
stopwords.update(["Mr", "S"])
wordcloud = WordCloud(stopwords=stopwords, max_words=300, width=1000, height=500, background_color="black").generate(text)

fig = px.imshow(wordcloud)
fig.update_layout(
    title=title,
    xaxis_visible=False,
    yaxis_visible=False,
)
fig.show('notebook')
